**This notebook is an exercise in the [Time Series](https://www.kaggle.com/learn/time-series) course.  You can reference the tutorial at [this link](https://www.kaggle.com/ryanholbrook/time-series-as-features).**

---


# Introduction #

Run this cell to set everything up!

In [ ]:
# Setup feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.time_series.ex4 import *

# Setup notebook
from pathlib import Path
from learntools.time_series.style import *  # plot style settings
from learntools.time_series.utils import plot_lags, make_lags, make_leads

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_log_error
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess


comp_dir = Path('../input/store-sales-time-series-forecasting')

store_sales = pd.read_csv(
    comp_dir / 'train.csv',
    usecols=['store_nbr', 'family', 'date', 'sales', 'onpromotion'],
    dtype={
        'store_nbr': 'category',
        'family': 'category',
        'sales': 'float32',
        'onpromotion': 'uint32',
    },
    parse_dates=['date'],
    infer_datetime_format=True,
)
store_sales['date'] = store_sales.date.dt.to_period('D')
store_sales = store_sales.set_index(['store_nbr', 'family', 'date']).sort_index()

family_sales = (
    store_sales
    .groupby(['family', 'date'])
    .mean() 
    .unstack('family')
    .loc['2017', ['sales', 'onpromotion']]
)

----------------------------------------------------------------------------
No todas las familias de productos tienen ventas que muestran un comportamiento cíclico, y tampoco la serie de ventas promedio. Las ventas de útiles escolares y de oficina, sin embargo, muestran patrones de crecimiento y decrecimiento que no están bien caracterizados por tendencias o temporadas. En esta pregunta y en la siguiente, modelará ciclos en las ventas de útiles escolares y de oficina utilizando características de retraso.

Tanto la tendencia como la estacionalidad crearán una dependencia serial que se muestra en los correlogramas y los diagramas de retraso. Para aislar cualquier comportamiento puramente *cíclico*, comenzaremos por desestacionalizar la serie. Utilice el código de la siguiente celda para desestacionalizar *Ventas de suministros*. Guardaremos el resultado en una variable `y_deseason`.

In [ ]:
supply_sales = family_sales.loc(axis=1)[:, 'SCHOOL AND OFFICE SUPPLIES']
y = supply_sales.loc[:, 'sales'].squeeze()

fourier = CalendarFourier(freq='M', order=4)
dp = DeterministicProcess(
    constant=True,
    index=y.index,
    order=1,
    seasonal=True,
    drop=True,
    additional_terms=[fourier],
)
X_time = dp.in_sample()
X_time['NewYearsDay'] = (X_time.index.dayofyear == 1)

model = LinearRegression(fit_intercept=False)
model.fit(X_time, y)
y_deseason = y - model.predict(X_time)
y_deseason.name = 'sales_deseasoned'

ax = y_deseason.plot()
ax.set_title("VENTAS DE MATERIAL ESCOLAR Y DE OFICINA(deseasonalized)");

¿Esta serie desestacionalizada muestra patrones cíclicos? Para confirmar nuestra intuición, podemos intentar aislar el comportamiento cíclico utilizando un gráfico de promedio móvil tal como lo hicimos con la tendencia. La idea es elegir una ventana lo suficientemente larga para suavizar la estacionalidad a corto plazo, pero lo suficientemente corta como para preservar los ciclos.

# 1) 
Ciclos de trazado

Cree un promedio móvil de siete días a partir de `y`, la serie de ventas de suministros. Usa una ventana centrada, pero no establezcas el argumento `min_periods`.

In [ ]:
# YOUR CODE HERE
y_ma = y.rolling(7, center=True).mean()

ax = y_ma.plot()
ax.set_title("Seven-Day Moving Average");


# Plot
ax = y_ma.plot()
ax.set_title("Promedio móvil de siete días");

# Check your answer
q_1.check()

In [ ]:
# Lines below will give you a hint or solution code
#q_1.hint()
#q_1.solution()

¿Ves cómo la trama de la media móvil se parece a la trama de la serie desestacionalizada? En ambos, podemos ver el comportamiento cíclico indicado.

-------------------------------------------------- --------------------------------------------

Examinemos nuestra serie desestacionalizada para determinar la dependencia serial. Eche un vistazo al correlograma de autocorrelación parcial y al diagrama de retardo.

In [ ]:
plot_pacf(y_deseason, lags=8);
plot_lags(y_deseason, lags=8, nrows=2);

# 2) Examinar la dependencia serial en *Ventas en tienda*

¿Alguno de los rezagos es significativo según el correlograma? ¿El diagrama de retardo sugiere alguna relación que no fuera evidente en el correlograma?



El correlograma indica que es probable que el primer retraso sea significativo, así como posiblemente el octavo retraso. El diagrama de retardo sugiere que el efecto es principalmente lineal.

In [ ]:
# View the solution (Run this cell to receive credit!)
#q_2.check()

-------------------------------------------------------------------------------

Recuerde del tutorial que un indicador principal es una serie cuyos valores en un momento pueden usarse para predecir el objetivo en un momento futuro: un indicador principal proporciona un "aviso anticipado" de los cambios en el objetivo.

El conjunto de datos de la competencia incluye una serie temporal que podría ser potencialmente útil como indicador principal: la serie en promoción, que contiene la cantidad de artículos en una promoción especial ese día. Dado que la propia empresa decide cuándo hacer una promoción, no hay que preocuparse por las "fugas anticipadas"; podríamos usar el valor en promoción del martes para pronosticar las ventas del lunes, por ejemplo.

Utilice la siguiente celda para examinar los valores adelantados y atrasados ​​de la promoción frente a las ventas de material escolar y de oficina.

In [ ]:
onpromotion = supply_sales.loc[:, 'onpromotion'].squeeze().rename('onpromotion')

# Drop days without promotions
plot_lags(x=onpromotion.loc[onpromotion > 1], y=y_deseason.loc[onpromotion > 1], lags=3, leads=3, nrows=1);

# 3) Examinar características de series de tiempo

¿Parece que los valores adelantados o atrasados ​​de `en promoción` podrían ser útiles como característica?

El gráfico de rezagos indica que tanto los valores adelantados como los rezagados de la promoción están correlacionados con las ventas de suministros. Esto sugiere que ambos tipos de valores podrían ser útiles como características. También puede haber algunos efectos no lineales.

In [ ]:
#q_3.check()

-------------------------------------------------------------------------------

# 4) Crear características de series temporales

Cree las características indicadas en la solución a la Pregunta 3. Si ninguna característica de esa serie sería útil, use un marco de datos vacío `pd.DataFrame()` como respuesta.

In [ ]:
# YOUR CODE HERE: Make features from `y_deseason`
X_lags = make_lags(y_deseason, lags=1)

X_promo = pd.concat([
    make_lags(onpromotion, lags=1),
    onpromotion,
    make_leads(onpromotion, leads=1),
], axis=1)

X = pd.concat([X_time, X_lags, X_promo], axis=1).dropna()
y, X = y.align(X, join='inner')



X = pd.concat([X_lags, X_promo], axis=1)
y, X = y.align(X, join='inner')

# Check your answer
q_4.check()

In [ ]:
# Lines below will give you a hint or solution code
#q_4.hint()
#q_4.solution()

Use the code in the next cell if you'd like to see predictions from the resulting model.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=30, shuffle=False)

model = LinearRegression(fit_intercept=False).fit(X_train, y_train)
y_fit = pd.Series(model.predict(X_train), index=X_train.index).clip(0.0)
y_pred = pd.Series(model.predict(X_valid), index=X_valid.index).clip(0.0)

rmsle_train = mean_squared_log_error(y_train, y_fit) ** 0.5
rmsle_valid = mean_squared_log_error(y_valid, y_pred) ** 0.5
print(f'Training RMSLE: {rmsle_train:.5f}')
print(f'Validation RMSLE: {rmsle_valid:.5f}')

ax = y.plot(**plot_params, alpha=0.5, title="Promedio de ventas", ylabel="Productos vendidos")
ax = y_fit.plot(ax=ax, label="Equipado", color='C0')
ax = y_pred.plot(ax=ax, label="Pronóstico", color='C3')
ax.legend();

-------------------------------------------------------------------------------

Los ganadores de los concursos de pronóstico de Kaggle a menudo incluyen promedios móviles y otras estadísticas móviles en sus conjuntos de características. Tales características parecen ser especialmente útiles cuando se usan con algoritmos GBDT como XGBoost.

En la Lección 2, aprendió a calcular promedios móviles para estimar tendencias. El cálculo de las estadísticas continuas que se utilizarán como características es similar, excepto que debemos tener cuidado para evitar la fuga anticipada. Primero, el resultado debe establecerse en el extremo derecho de la ventana en lugar del centro; es decir, debemos usar center=False (el valor predeterminado) en el método de balanceo. En segundo lugar, el objetivo debe retrasarse un paso.

# 5) Crear características estadísticas

Edite el código en la siguiente celda para crear las siguientes funciones:
- Mediana móvil de 14 días ('mediana') del objetivo rezagado
- Desviación estándar móvil de 7 días (`std`) del objetivo rezagado
- Suma de 7 días (`sum`) de artículos "en promoción", con ventana centrada

In [ ]:
y_lag = supply_sales.loc[:, 'sales'].shift(1)
onpromo = supply_sales.loc[:, 'onpromotion']

y_lag = supply_sales.loc[:, 'sales'].shift(1)
onpromo = supply_sales.loc[:, 'onpromotion']

mean_7 = y_lag.rolling(7).mean()
median_14 = y_lag.rolling(14).median()
std_7 = y_lag.rolling(7).std()
promo_7 = onpromo.rolling(7, center=True).sum()


# Check your answer
#q_5.check()

In [ ]:
# Lines below will give you a hint or solution code
#q_5.hint()
#q_5.solution()

Check out the Pandas [`Window` documentation](https://pandas.pydata.org/pandas-docs/stable/reference/window.html) for more statistics you can compute. Also try "exponential weighted" windows by using `ewm` in place of `rolling`; exponential decay is often a more realistic representation of how effects propagate over time.

# Keep Going #

[**Create hybrid forecasters**](https://www.kaggle.com/ryanholbrook/hybrid-models) and combine the strengths of two machine learning algorithms.

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/time-series/discussion) to chat with other learners.*